In [22]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
alchemyEngine=create_engine('postgresql://desmondmolloy:@127.0.0.1', pool_recycle=3600);
dbConnection=alchemyEngine.connect()
df=pd.read_sql("SELECT SUM(daily_vaccinations_per_million) AS vaccinations, country FROM vaccine WHERE daily_vaccinations_per_million IS NOT NULL GROUP BY country ORDER BY vaccinations DESC;", dbConnection)
secondf=pd.read_sql("WITH best_table AS (SELECT daily_vaccinations, country, date, ROW_NUMBER() OVER(PARTITION BY country ORDER BY country,daily_vaccinations DESC) AS RowNumber FROM vaccine WHERE \"daily_vaccinations\" IS NOT NULL)\
select CAST(date AS VARCHAR) from  best_table where RowNumber=1;", dbConnection)
myjsobject="export var country={}; export var vaccines={}; export var dates={};".format(list(df['country']),list(df['vaccinations']),list(secondf['date']))
dest_file='/Users/desmondmolloy/Downloads/Repositories/example-react-app/src/data/test_content.js'
text_file = open(dest_file, "wt")
n = text_file.write(myjsobject)
text_file.close()
secondf.head()

,date
0,2020-12-30
1,2021-01-21
2,2020-12-24
3,2021-01-18
4,2021-01-23


In [ ]:
df.columns = ["y", "label"]
ajson=df.to_json(orient='records')
import json
with open('/Users/desmondmolloy/Desktop/Codecademy_bench/src/data/response_time.json', 'w') as outfile:
    json.dump(ajson, outfile)
#allowed values are: {‘split’, ‘records’, ‘index’, ‘table’}.
ajson
myjsobject="export var newData={}".format(ajson)
dest_file='/Users/desmondmolloy/Desktop/Codecademy_bench/src/data/test.js'
text_file = open(dest_file, "wt")
n = text_file.write(myjsobject)
text_file.close()

In [1]:
def column_outwriter(df):
    collist=list(df.columns)
    return_json=""
    for colname in collist:
        stepjson="export var {}={}; ".format(colname, list(df[colname]))
        return_json=return_json+stepjson
    return return_json

def sql_to_js_file(query,filepath):
    import pandas as pd
    import numpy as np
    from sqlalchemy import create_engine
    alchemyEngine=create_engine('postgresql://desmondmolloy:@127.0.0.1', pool_recycle=3600);
    dbConnection=alchemyEngine.connect()
    df=pd.read_sql(query, dbConnection)
    myjsobject=column_outwriter(df)
    text_file = open(filepath, "wt")
    n = text_file.write(myjsobject)
    text_file.close()


In [4]:
sql_to_js_file("WITH overtime_count AS(SELECT COUNT(*) as total_overdue, neighborhood AS overdue_neighborhood FROM service_requests WHERE ontime='OVERDUE' GROUP BY neighborhood), \
full_count AS(SELECT COUNT(*) AS total, neighborhood FROM service_requests GROUP BY neighborhood), \
united_table AS (SELECT * FROM overtime_count \
LEFT JOIN full_count ON overtime_count.overdue_neighborhood=full_count.neighborhood) \
SELECT neighborhood, total_overdue*total/POWER(total,2) AS percent_overdue FROM united_table WHERE total is not null ORDER BY percent_overdue DESC;","src/data/overdue_percent.js")
sql_to_js_file("WITH cte AS(SELECT DATE_PART('day', closed_dt - open_dt) AS duration, neighborhood FROM service_requests WHERE closed_dt IS NOT NULL) \
SELECT AVG(duration) AS neighborhood_mean, neighborhood AS neighborhood_for_time FROM cte WHERE neighborhood IS NOT NULL GROUP BY neighborhood ORDER BY neighborhood_mean DESC;","src/data/response_time.js")